# Corn Grain Yield Response to Nitrogen Rates
---

**Name**: Adrian Correndo

**Semester**: Spring 2019

**Project area**: Agronomy

## **Table of contents**
1. [DESCRIPTION](#description)

2. [CODE:](#Code)
    a. [Modules and data](#modules_datafile)
    b. [Sorting by](#Sort)
    c. [Grouping by TRIAL](#Group)
    d. [Grain yield of non-fertilized plots](#Y0)
    e. [Maximum grain yield](#Ymax)
    f. [Maximum yield response to N](#NRmax)
    g. [Nitrogen response](#NR)
    h. [Nitrogen agronomic efficiency](#NAE)
    i. [Non-fertilized plots](#N0_plots)
    j. [Fertilized plots](#Nf_plots)
    
3. [OUTPUTS:](#OUTPUTS)
    a. [N0 plots](#N0_file)
    b. [Nf plots](#Nf_file)
    
4. [SOIL TEXTURE:](#STx)
    a. [Classes and Frequencies](#STx_freq)
    b. [Grouping by Soil texture](#GSTx)
    c. [Descriptive Stats by STx groups](#Stats)
    
5. [BONUS TRACK:](#BT)
    a. [Interactive Map with location clusters](#Map)   

<a name="description"></a>
## **1. DESCRIPTION**
## Objective

Automating the calculation of grain yield (GY) response to different rates of nitrogen (N) fertilizer in corn (*Zea Mays* L.), and related fertilizer use efficiencies (NAE) in a database of more than one thousand experiments with different designs. A secondary goal is to explore descriptive statistics of variables of interest grouping experiments by soil texture classes (STx).

## Inputs

*.csv file with 4 columns: TRIAL, STx, N rate, and GY, where:

**-TRIAL**: Experiment ID number;
**-STx**: Soil texture class of typical pedon  (Soil Survey Staff, 2018);
**-Nrate**: Nitrogen rate (kg N / ha);
**-GY**: Grain Yield when Nrate=0  (Mg / ha, 15.5% moisture);

## Outcomes

Two "*.csv" files

1. N0_plots.csv, with the following columns: TRIAL, STx, Y0, Ymax, NRmax, NRmax_r, NAEmax.
2. Nf_plots.csv, with the following columns: TRIAL, STx, Nrate, GY, Y0, Ymax, NR, NRr and NAE, where:

**-Y0**: GY when Nrate=0  (Mg / ha);
**-Ymax**: maximum observed GY (Mg / ha);
**-NR**: absolute nitrogen response corresponding to each fertilizer rate different from 0  (Mg / ha).
**-NRr**: relative nitrogen response corresponding to each fertilizer rate different from 0  (%).
**-NRmax**: maximum absolute nitrogen response (Mg / ha).
**-NRmax_r**: maximum relative nitrogen response (%).
**-NAE**: nitrogen agronomic efficiency as NR divided by its corresponding Nrate (kg NR / kg applied N).

MAIN challenges were related to: 

i) the # of Nrate levels and the ammount of applied N (kg) vary across trials;

ii) Y0 and Ymax values take place at **Trial** level, while the NR and NAE values, at a sub-level by a given **Trial-Nrate combination**.

## Rationale
I'm working on a review paper for which I collected more than 1200 experiments. Automating these calculations will save me a significant amount of time and avoid potential misscalculations when processing the data.

<a name="Code"></a>
## **2. CODE**

For runnig the code a random sample of 80 trials has been taken from the original database, and it's provided in the file named as "correndo_data.csv".

<a name="modules_datafile"></a>
### 2. a. Modules and datafile

In [22]:
# Setting working directory and navigating
import os
import glob
import pandas as pd

loc = os.path.join(glob.os.getcwd())#Defining the current directory to work
dataset = pd.read_csv('example_dataset.csv', encoding='latin-1')#Reading the file in the cd
df = pd.DataFrame(dataset)
df.to_csv('example_dataset.csv')
df.head()

,TRIAL,STx,Nrate,GY
0,1,silty clay loam,0,13.317
1,1,silty clay loam,84,14.434
2,1,silty clay loam,140,15.267
3,1,silty clay loam,196,15.405
4,1,silty clay loam,280,15.496


<a name="Sort"></a>
### 2. b. Sorting by "TRIAL" and by "Nrate"(ascending)

In [23]:
sdf = df.sort_values(['TRIAL', 'Nrate'],ascending=True) #sorted dataframe (sdf) not modifying the original df
sdf = pd.DataFrame(sdf)
sdf.head(10)

,TRIAL,STx,Nrate,GY
0,1,silty clay loam,0,13.317
1,1,silty clay loam,84,14.434
2,1,silty clay loam,140,15.267
3,1,silty clay loam,196,15.405
4,1,silty clay loam,280,15.496
5,4,silt loam,0,9.742
6,4,silt loam,84,11.287
7,4,silt loam,140,11.037
8,4,silt loam,196,10.963
9,4,silt loam,280,11.473


<a name="Group"></a>
### 2. c. Grouping by TRIAL

In [24]:
# Defining "trials" element as a group of rows corresponding to the same TRIAL. "Groupby" function of Pandas
trials = sdf.groupby("TRIAL")

<a name="Y0"></a>
### 2. d. Grain yield of non-fertilized plots (Y0)

In [25]:
#Using "nth" function of Pandas to identify -within 'trial'- a row position based on Nrate value (in this case Nrate == 0)
sdf_2 = sdf.join(trials['GY'].nth(trials['Nrate']==0).rename('Y0'), 'TRIAL')
sdf_2.head()

,TRIAL,STx,Nrate,GY,Y0
0,1,silty clay loam,0,13.317,13.317
1,1,silty clay loam,84,14.434,13.317
2,1,silty clay loam,140,15.267,13.317
3,1,silty clay loam,196,15.405,13.317
4,1,silty clay loam,280,15.496,13.317


<a name="Ymax"></a>
### 2. e. Maximum grain yield (Ymax)

In [26]:
#Using "max" function of Pandas to identify -within each trial- the row position based on GY value (in this case GY == max)
sdf_3 = sdf_2.join(trials['GY'].max().rename('Ymax'),'TRIAL')
sdf_3.head()

,TRIAL,STx,Nrate,GY,Y0,Ymax
0,1,silty clay loam,0,13.317,13.317,15.496
1,1,silty clay loam,84,14.434,13.317,15.496
2,1,silty clay loam,140,15.267,13.317,15.496
3,1,silty clay loam,196,15.405,13.317,15.496
4,1,silty clay loam,280,15.496,13.317,15.496


<a name="NRmax"></a>
### 2. f. Maximum yield response to nitrogen (as Ymax - Y0; absolute and relative)

In [27]:
for i in sdf_3:
    NRmax = (sdf_3.Ymax - sdf_3.Y0).round(3)
    NRmax_r = (((sdf_3.Ymax - sdf_3.Y0) / sdf_3.Y0)*100).round(3)
sdf_4 = sdf_3.join(NRmax.rename('NRmax'))
sdf_4 = sdf_4.join(NRmax_r.rename('NRmax_r'))
sdf_4.head()

,TRIAL,STx,Nrate,GY,Y0,Ymax,NRmax,NRmax_r
0,1,silty clay loam,0,13.317,13.317,15.496,2.179,16.363
1,1,silty clay loam,84,14.434,13.317,15.496,2.179,16.363
2,1,silty clay loam,140,15.267,13.317,15.496,2.179,16.363
3,1,silty clay loam,196,15.405,13.317,15.496,2.179,16.363
4,1,silty clay loam,280,15.496,13.317,15.496,2.179,16.363


<a name="NR"></a>
### 2. g. Nitrogen response (at each rate, absolute and relative)

In [28]:
for i in sdf_4:
    NR = sdf_4.GY - sdf_4.Y0
    sdf_5 = sdf_4.join(NR.rename('NR'))
    NRr = (NR / sdf_5.Y0)*100
    sdf_5 = sdf_5.join(NRr.rename('NRr'))
        
sdf_5.head()

,TRIAL,STx,Nrate,GY,Y0,Ymax,NRmax,NRmax_r,NR,NRr
0,1,silty clay loam,0,13.317,13.317,15.496,2.179,16.363,0.000,0.000000
1,1,silty clay loam,84,14.434,13.317,15.496,2.179,16.363,1.117,8.387775
2,1,silty clay loam,140,15.267,13.317,15.496,2.179,16.363,1.950,14.642938
3,1,silty clay loam,196,15.405,13.317,15.496,2.179,16.363,2.088,15.679207
4,1,silty clay loam,280,15.496,13.317,15.496,2.179,16.363,2.179,16.362544


<a name="NAE"></a>
### 2. h. Nitrogen agronomic efficiency

In [29]:
for i in sdf_5:
    NAE = ((sdf_5.NR)*1000 / sdf_5.Nrate)  
sdf_6 = sdf_5.join(NAE.round(3).rename('NAE'))

sdf_6.head()

,TRIAL,STx,Nrate,GY,Y0,Ymax,NRmax,NRmax_r,NR,NRr,NAE
0,1,silty clay loam,0,13.317,13.317,15.496,2.179,16.363,0.000,0.000000,NaN
1,1,silty clay loam,84,14.434,13.317,15.496,2.179,16.363,1.117,8.387775,13.298
2,1,silty clay loam,140,15.267,13.317,15.496,2.179,16.363,1.950,14.642938,13.929
3,1,silty clay loam,196,15.405,13.317,15.496,2.179,16.363,2.088,15.679207,10.653
4,1,silty clay loam,280,15.496,13.317,15.496,2.179,16.363,2.179,16.362544,7.782


<a name="NAEmax"></a>
### 2. i. Maximum NAE

In [30]:
# Grouping by TRIAL again (now with all the columns of interest)
trials_2 = sdf_6.groupby("TRIAL")

In [31]:
# Adding the new variable as a column'NAEmax'
sdf_7 = sdf_6.join(trials_2['NAE'].max().rename('NAEmax'),'TRIAL')
sdf_7.head()

,TRIAL,STx,Nrate,GY,Y0,Ymax,NRmax,NRmax_r,NR,NRr,NAE,NAEmax
0,1,silty clay loam,0,13.317,13.317,15.496,2.179,16.363,0.000,0.000000,NaN,13.929
1,1,silty clay loam,84,14.434,13.317,15.496,2.179,16.363,1.117,8.387775,13.298,13.929
2,1,silty clay loam,140,15.267,13.317,15.496,2.179,16.363,1.950,14.642938,13.929,13.929
3,1,silty clay loam,196,15.405,13.317,15.496,2.179,16.363,2.088,15.679207,10.653,13.929
4,1,silty clay loam,280,15.496,13.317,15.496,2.179,16.363,2.179,16.362544,7.782,13.929


<a name="N0_plots"></a>
### 2. j. Non-fertilized plots

In [32]:
# Filtering by Nrate = 0
N0_plots = pd.DataFrame(sdf_7[sdf_7.Nrate == 0])
# Dropping columns we won't use in this df
N0_plots = N0_plots.drop(columns=['Nrate', 'GY', 'NR','NRr', 'NAE'])
N0_plots.head()

,TRIAL,STx,Y0,Ymax,NRmax,NRmax_r,NAEmax
0,1,silty clay loam,13.317,15.496,2.179,16.363,13.929
5,4,silt loam,9.742,11.473,1.731,17.768,18.393
10,5,silt loam,10.969,13.638,2.669,24.332,12.721
15,13,silty clay,7.903,13.671,5.768,72.985,76.321
20,29,silt loam,10.330,15.628,5.298,51.288,47.304


<a name="Nf_plots"></a>
### 2. h. Fertilized plots

In [33]:
# Filtering by Nrate > 0
Nf_plots = pd.DataFrame(sdf_7[sdf_7.Nrate > 0])
# Dropping columns we won't use in this df
Nf_plots = Nf_plots.drop(columns=['NRmax','NRmax_r', 'NAEmax'])
Nf_plots.head()

,TRIAL,STx,Nrate,GY,Y0,Ymax,NR,NRr,NAE
1,1,silty clay loam,84,14.434,13.317,15.496,1.117,8.387775,13.298
2,1,silty clay loam,140,15.267,13.317,15.496,1.950,14.642938,13.929
3,1,silty clay loam,196,15.405,13.317,15.496,2.088,15.679207,10.653
4,1,silty clay loam,280,15.496,13.317,15.496,2.179,16.362544,7.782
6,4,silt loam,84,11.287,9.742,11.473,1.545,15.859166,18.393


<a name="OUTPUTS"></a>
## 3. FILE OUTPUTS

<a name="N0_plots_file"></a>
### 3. a. File of N0 plots

In [34]:
N0_plots.to_csv('N0_plots.csv')

<a name="Nf_plots_file"></a>
### 3. b. File of Nf plots

In [35]:
Nf_plots.to_csv('Nf_plots.csv')

<a name="STx"></a>
## 4. SOIL TEXTURE

<a name="STx_freq"></a>
### 4. a. How many unique classes of soil texture are in the database?

In [36]:
# Using the N0_plots table (1 row per TRIAL)
text_class = N0_plots['STx'].unique().tolist() 
text_freq = N0_plots['STx'].value_counts()
print("The number of different soil texture classes is:", len(text_class))

The number of different soil texture classes is: 8


#### New df with Soil texture classes and frequencies (counts)

In [37]:
STx = pd.DataFrame(data=({'Frequency': text_freq}))
# File
STx.to_csv('STx.csv')
STx

,Frequency
silt loam,26
loam,18
silty clay loam,13
clay loam,7
sandy loam,7
silty clay,5
loamy sand,2
clay,2


<a name="GSTx"></a>
### 4. b. Grouping by Soil Texture

In [39]:
STxs = N0_plots.groupby("STx")

<a name="Stats"></a>
### 4. c. Descriptive stats by Soil Texture

<a name="Y0_stats"></a>
#### Y0 stats

In [40]:
Y0_stats = pd.DataFrame(STxs.Y0.describe())
Y0_stats = Y0_stats.apply(lambda x: round(x, 2))
Y0_stats

,count,mean,std,min,25%,50%,75%,max
STx,,,,,,,,
clay,2.0,8.34,3.38,5.96,7.15,8.34,9.53,10.73
clay loam,7.0,5.69,4.00,2.03,2.73,5.11,6.97,13.33
loam,18.0,7.59,3.65,1.04,4.90,6.75,9.85,14.91
loamy sand,2.0,3.26,0.34,3.01,3.14,3.26,3.38,3.50
sandy loam,7.0,6.69,3.84,1.78,3.51,7.10,9.54,11.83
silt loam,26.0,7.80,3.39,2.13,4.71,8.08,10.33,13.86
silty clay,5.0,8.22,2.37,4.33,7.90,8.80,9.65,10.40
silty clay loam,13.0,9.67,3.23,5.60,7.16,8.70,12.05,16.11


<a name="Ymax_stats"></a>
#### Ymax stats

In [41]:
Ymax_stats = pd.DataFrame(STxs.Ymax.describe())
Ymax_stats = Ymax_stats.apply(lambda x: round(x, 2))
Ymax_stats

,count,mean,std,min,25%,50%,75%,max
STx,,,,,,,,
clay,2.0,10.89,1.27,10.00,10.44,10.89,11.34,11.79
clay loam,7.0,9.92,3.25,6.26,7.25,9.92,12.24,14.27
loam,18.0,12.07,3.14,6.67,10.02,12.38,14.03,18.56
loamy sand,2.0,7.93,6.74,3.16,5.55,7.93,10.32,12.70
sandy loam,7.0,10.50,4.16,4.10,9.16,10.02,11.54,17.99
silt loam,26.0,12.21,3.50,5.08,10.71,13.15,14.58,17.21
silty clay,5.0,12.35,1.59,9.70,12.32,12.57,13.49,13.67
silty clay loam,13.0,12.68,2.25,8.67,11.10,12.62,13.38,17.04


<a name="NRmax_stats"></a>
#### NRmax stats

In [42]:
NRmax_stats = pd.DataFrame(STxs.NRmax.describe())
NRmax_stats = NRmax_stats.apply(lambda x: round(x, 2))
NRmax_stats

,count,mean,std,min,25%,50%,75%,max
STx,,,,,,,,
clay,2.0,2.55,2.11,1.06,1.81,2.55,3.30,4.04
clay loam,7.0,4.22,2.63,0.94,2.50,3.86,5.58,8.60
loam,18.0,4.48,2.15,1.07,3.41,4.38,5.55,9.85
loamy sand,2.0,4.68,6.40,0.15,2.41,4.68,6.94,9.20
sandy loam,7.0,3.81,2.29,1.40,2.00,2.70,5.93,6.74
silt loam,26.0,4.41,2.74,0.52,2.58,3.90,5.57,10.16
silty clay,5.0,4.13,2.83,0.90,2.16,3.83,5.77,7.99
silty clay loam,13.0,3.01,2.28,0.57,1.40,2.58,3.10,7.70


<a name="NAEmax_stats"></a>
#### NAEmax stats

In [43]:
NAEmax_stats = pd.DataFrame(STxs.NAEmax.describe())
NAEmax_stats = NAEmax_stats.apply(lambda x: round(x, 2))
NAEmax_stats

,count,mean,std,min,25%,50%,75%,max
STx,,,,,,,,
clay,2.0,17.75,18.83,4.43,11.09,17.75,24.40,31.06
clay loam,7.0,46.64,22.73,12.24,36.62,39.79,58.99,83.24
loam,18.0,36.59,16.79,11.63,21.62,35.22,50.17,67.94
loamy sand,2.0,31.74,42.22,1.89,16.82,31.74,46.67,61.60
sandy loam,7.0,27.04,20.23,7.20,14.21,20.15,34.51,64.48
silt loam,26.0,39.19,24.77,2.15,23.63,36.72,51.95,106.58
silty clay,5.0,40.22,30.09,8.96,9.68,46.87,59.27,76.32
silty clay loam,13.0,29.24,22.60,5.75,13.93,16.42,41.48,79.10


<a name="BT"></a>
## 5. BONUS TRACK

In [45]:
# Import FOLIUM library for creating the MAP, we'll also need Pandas and Numpy

import folium # need to install it
import pandas as pd
import numpy as np
import folium.plugins
folium.plugins.MarkerCluster()
from folium.plugins import MarkerCluster # This is the Cluster function for the map

In [50]:
# Subdata set with random locations from KS, NE, IA and IL
data = pd.read_csv('loc_coords.csv')
data = pd.DataFrame(data)
data.head()

,lat,lon
0,36.5953,-101.6366
1,37.3638,-95.2876
2,37.3638,-95.2876
3,37.4259,-88.6642
4,37.4259,-88.6642


In [52]:
#Map with Clusters
map = folium.Map(location=[40, -95], tiles="Mapbox Bright", zoom_start=4.5) # Creating the map, centered at [40,95], tiles indicates the type of map
marker_cluster = MarkerCluster().add_to(map)
locations = data[['lat', 'lon']]
locationslist = locations.values.tolist()
for i in range(0, len(locationslist)):
    folium.Marker(locationslist[i]).add_to(marker_cluster) #if you have metadata could be displayed using "popup=data["metadata_variable"][i]""
    
map.save('map.html')  
map